# Regular Genomics Project **RNA Localisation**


---

**Problem definition**:

TODO


### **1.** Data Preparation

Firstly, we import several necessary packages and load in our data:

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold

In [2]:
# Initializing test/train split
np.random.seed(3)
data_org = pd.read_csv('~/Downloads/final_data.csv')
test_data = data_org.sample(frac=0.1)
train_data = data_org.drop(test_data.index)


### **2** |  Initializations

We use as baseline model the [RNATracker](https://github.com/HarveyYan/RNATracker/blob/master/Models/cnn_bilstm_attention.py) model and a CNN model.


In [3]:
max_seq_len = train_data['seq'].apply(lambda x: len(x)).max()

# DataLoader
param_dataLoader_train = {
    'padding_length': max_seq_len,
    "batch_size_train": 32,
    "shuffle_batch_train": True
    }

param_dataLoader_valid = {
    'padding_length': max_seq_len,
    "batch_size_valid": 32,
    "shuffle_batch_valid": True
    }

# KFold
param_KFold = {
    "n_splits": 5,
    "shuffle": True
    }

# Model Parameter
param_model = {
    #'optimizer': 'adam'
    #'loss':
    #'metrics':
}

In [7]:
# Splitting for 5fold

kf = KFold(**param_KFold)
folds = kf.split(train_data)

In [11]:
# training, only need for the model initialization to change in general
from CNN import CNN

VALIDATION_ACCURACY = []
VALIDATION_LOSS = []

for i, (train_split, valid_split) in enumerate(folds):
    model = CNN(architecure='dcmcdmcmfe', 
                dropouts=[.2, .2], 
                pooling=[2, 2, 2], 
                conv=[{'filters': 64, 'kernel_size': 3, 'activation': 'relu', 'input_shape': (train_data['seq'].apply(lambda x: len(x)).max(), 4)},
                      {'filters': 64, 'kernel_size': 3, 'activation': 'relu', 'padding': 'same'},
                      {'filters': 64, 'kernel_size': 3, 'activation': 'relu', 'padding': 'same'}],
                dense=[{'units': 9, 'activation': 'softmax'}],
                **param_model )     

    model.fit(train_data=train_data.iloc[train_split], **param_dataLoader_train)
    results = model.evaluate(eval_data=train_data.iloc[valid_split], **param_dataLoader_valid)
    results = dict(zip(model.model.metrics_names, results))

    VALIDATION_ACCURACY.append(results['accuracy'])
    VALIDATION_LOSS.append(results['loss'])

78/78 [==============================] - 54s 694ms/step - loss: 2.1767 - accuracy: 0.1980


In [14]:
test_result = model.evaluate(test_data, **param_dataLoader_valid)
result = dict(zip(model.model.metrics_names, test_result))
TEST_ACCURACY = result['accuracy']
TEST_LOSS = result['loss']

44/44 [==============================] - 28s 630ms/step - loss: 2.1774 - accuracy: 0.1789


In [12]:
VALIDATION_ACCURACY

[0.19710378348827362, 0.20836685597896576, 0.19798792898654938]

In [13]:
VALIDATION_LOSS

[2.177325487136841, 2.1693270206451416, 2.1767470836639404]

## DeepRNALoc



In [ ]:
from DeepmRNALoc import build_model
from GeneDataLoader import GeneDataLoader


# Define the training parameters
epochs = 10
batch_size = 32

# Create an instance of the GeneDataLoader
data_loader = GeneDataLoader(data_table=train_data, padding_length=max_seq_len)
max_len = data_loader.max_len

# Create the model
model = build_model(max_len)

# TODO: fold the dataset and look 

# Train the model
model.fit(data_loader, epochs=epochs, verbose=0)

# Evaluate the model
loss, accuracy = model.evaluate(data_loader)
print("Loss:", loss)
print("Accuracy:", accuracy)

2023-06-18 17:04:15.669567: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-18 17:04:29.077624: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-18 17:04:29.080593: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split